In [1]:
import pandas as pd

# --- 1. CONFIGURAZIONE E CARICAMENTO ---
# Definiamo il percorso del file. Se il file è nella stessa cartella dello script, basta il nome.
file_path = 'data/train/training_data.csv'

print(f"Sto caricando il file da: {file_path}...")
df = pd.read_csv(file_path)
print("Caricamento completato!")

# --- 2. ORDINAMENTO TEMPORALE (CRUCIALE) ---
# I dati grezzi potrebbero non essere sequenziali. 
# Per fare prognostica, dobbiamo garantire che la riga N venga prima della riga N+1 nel tempo.
# Ordiniamo per:
# 1. ESN (Numero Motore): Per non mischiare i dati del motore 101 con il 102.
# 2. Cycles_Since_New (Età): Per mettere in fila i voli dal primo all'ultimo.
# 3. Snapshot (Istante): Per mettere in ordine gli scatti all'interno dello stesso volo.
df = df.sort_values(by=['ESN', 'Cycles_Since_New', 'Snapshot'])

# Resettiamo l'indice per avere una numerazione pulita delle righe (0, 1, 2...)
df = df.reset_index(drop=True)

# --- 3. ISPEZIONE DEI DATI ---
print("\n--- INFO GENERALE DATASET ---")
# .info() è utilissimo: ci dice quante righe totali abbiamo e il tipo di dati.
df.info()

print("\n--- ANTEPRIMA (PRIME 5 RIGHE) ---")
# Vediamo concretamente come sono fatti i dati
print(df.head())

# --- 4. CONTROLLO QUALITÀ DATI (MISSING VALUES) ---
print("\n--- CONTEGGIO VALORI MANCANTI (NaN) ---")
# Contiamo quanti dati mancano per ogni colonna.
# Se vedi numeri > 0 qui, dovrai gestire questi buchi nella fase di pulizia.
null_counts = df.isnull().sum()
print(null_counts[null_counts > 0]) # Stampa solo le colonne che hanno problemi

Sto caricando il file da: data/train/training_data.csv...
Caricamento completato!

--- INFO GENERALE DATASET ---
<class 'pandas.DataFrame'>
RangeIndex: 59702 entries, 0 to 59701
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ESN                 59702 non-null  int64  
 1   Cycles_Since_New    59702 non-null  int64  
 2   Snapshot            59702 non-null  int64  
 3   Cumulative_WWs      59702 non-null  int64  
 4   Cumulative_HPC_SVs  59702 non-null  int64  
 5   Cumulative_HPT_SVs  59702 non-null  int64  
 6   Sensed_Altitude     59702 non-null  float64
 7   Sensed_Mach         59702 non-null  float64
 8   Sensed_Pamb         59702 non-null  float64
 9   Sensed_Pt2          59702 non-null  float64
 10  Sensed_TAT          59702 non-null  float64
 11  Sensed_WFuel        59005 non-null  float64
 12  Sensed_VAFN         59702 non-null  float64
 13  Sensed_VBV          59702 non-null  float64
 14  

In [4]:
#--
print("Analisi snapshot")

# Count ci dice quante righe (snapshot) ci sono per ogni volo
snapshot_counts = df.groupby(['ESN', 'Cycles_Since_New'])['Snapshot'].count()

# 2. Statistiche di base
max_snap = snapshot_counts.max()
min_snap = snapshot_counts.min()
print(f"Max snapshot per volo: {max_snap}")
print(f"Min snapshot per volo: {min_snap}")

# 3. Vediamo la distribuzione
# Ci dice: "Ci sono X voli con 8 snapshot, Y voli con 7 snapshot..."
print("\nDistribuzione frequenza snapshot:")
print(snapshot_counts.value_counts().sort_index())

# 4. Identificazione dei voli con "buchi" (Gaps)
# Esempio: ho snapshot 1, 2, 4 -> Manca il 3. Questo codice lo trova.
def has_gap(series):
    # Crea un set dei numeri attesi (dal minimo al massimo trovati in quel volo)
    expected = set(range(series.min(), series.max() + 1))
    actual = set(series)
    # Se la differenza tra i due set non è vuota, manca qualcosa
    return len(expected - actual) > 0

# Applichiamo la funzione a tutti i voli (può richiedere qualche secondo)
gaps = df.groupby(['ESN', 'Cycles_Since_New'])['Snapshot'].apply(has_gap)
num_gaps = gaps[gaps == True].count()

print(f"\nNumero di voli con sequenza snapshot interrotta (buchi): {num_gaps}")

if num_gaps > 0:
    print("Esempio di volo con buchi:")
    # Prendiamo il primo caso trovato
    example_idx = gaps[gaps == True].index[0] 
    print(f"Motore {example_idx[0]}, Volo {example_idx[1]}")
    print(df[(df['ESN'] == example_idx[0]) & (df['Cycles_Since_New'] == example_idx[1])][['Snapshot', 'Sensed_Altitude']])

Analisi snapshot
Max snapshot per volo: 8
Min snapshot per volo: 7

Distribuzione frequenza snapshot:
Snapshot
7    4330
8    3674
Name: count, dtype: int64

Numero di voli con sequenza snapshot interrotta (buchi): 4330
Esempio di volo con buchi:
Motore 101, Volo 0
   Snapshot  Sensed_Altitude
0         1       851.321516
1         2       903.321516
2         3      3410.321516
3         4     20259.321516
4         6     21991.321516
5         7     21997.321516
6         8     21996.321516
